In [1]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import collections
import csv
import random
import pickle
from torch.utils.data import DataLoader
import dgl
import pandas as pd

class Subgraphs(Dataset):


    def __init__(self, root, mode, subgraph_list, subgraph2label, subgraph2center_node):   

        self.subgraph2label = subgraph2label
        self.subgraph_list = subgraph_list
        self.subgraph2center_node = subgraph2center_node
        self.data = pd.read_csv(os.path.join(root, mode + '.csv'))  # csv path
    
    def __getitem__(self, index):
        
        return self.subgraph_list[self.data.iloc[index]['name']], self.subgraph2label[self.data.iloc[index]['name']], self.subgraph2center_node[self.data.iloc[index]['name']] 

    def __len__(self):
        # as we have built up to batchsz of sets, you can sample some small batch size of sets.
        return len(self.data)
    
def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels, center_nodes = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.LongTensor(labels), torch.LongTensor(center_nodes)


In [2]:
class Subgraphs_test(Dataset):


    def __init__(self, root, mode, subgraph_list, subgraph2label, subgraph2center_node, k_shot, n_qry):   

        self.subgraph2label = subgraph2label
        self.subgraph_list = subgraph_list
        self.subgraph2center_node = subgraph2center_node
        
        self.data = pd.read_csv(os.path.join(root, mode + '.csv'))  # csv path
        self.k_shot = k_shot
        self.n_qry = n_qry
        
        self.labels = np.unique(self.data.label.values)
        self.labels_dict = dict(zip(list(self.labels), list(range(len(self.labels)))))
        
    def __getitem__(self, index):
        support = []
        query = []
        
        for i in self.labels:
            df_labels = self.data[self.data.label == i].reset_index(drop = True)
            support = support + list(df_labels.sample(n = self.k_shot)['name'].values)
            query = query + list(df_labels[~df_labels.name.isin(support)].sample(n = self.n_qry)['name'].values)
        
        return [self.subgraph_list[i] for i in support], [self.labels_dict[self.subgraph2label[i]] for i in support], [self.subgraph2center_node[i] for i in support], [self.subgraph_list[i] for i in query], [self.labels_dict[self.subgraph2label[i]] for i in query], [self.subgraph2center_node[i] for i in query]

    def __len__(self):
        # 100 "batches"
        return 100
    
def collate_test(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    
    graphs, labels, center_nodes, g_qry, l_qry, c_qry = map(list, zip(*samples))
    batched_graph_spt = dgl.batch(graphs[0])
    batched_graph_qry = dgl.batch(g_qry[0])
    
    return batched_graph_spt, torch.LongTensor(labels), torch.LongTensor(center_nodes), batched_graph_qry, torch.LongTensor(l_qry), torch.LongTensor(c_qry)


In [3]:
import dgl.function as fn
import torch
import torch.nn as nn


# Sends a message of node feature h.
msg = fn.copy_src(src='h', out='m')

def reduce(nodes):
    """Take an average over all neighbor node features hu and use it to
    overwrite the original node feature."""
    accum = torch.mean(nodes.mailbox['m'], 1)
    return {'h': accum}

class NodeApplyModule(nn.Module):
    """Update the node feature hv with ReLU(Whv+b)."""
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        # Initialize the node features with h.
        g.ndata['h'] = feature
        g.update_all(msg, reduce)
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [4]:
import copy

In [5]:
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()

        self.layers = nn.ModuleList([
            GCN(in_dim, hidden_dim, F.relu),
            GCN(hidden_dim, hidden_dim, F.relu)])
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g, to_fetch, features):
        # For undirected graphs, in_degree is the same as
        # out_degree.
        #h = g.in_degrees().view(-1, 1).float().to(device)
        #h = torch.tensor([1.]*g.number_of_nodes()).view(-1, 1).float()
        #h = torch.tensor(features)
        h = features.float()
        for conv in self.layers:
            h = conv(g, h)
        g.ndata['h'] = h
        #print(h.shape)
        #hg = dgl.mean_nodes(g, 'h')
        #print(to_fetch)
        num_nodes_ = g.batch_num_nodes
        num_nodes_.insert(0, 0)
        offset = torch.cumsum(torch.LongTensor(num_nodes_), dim = 0)[:-1].to(device)
        hg = h[to_fetch + offset]
        #print(hg.shape)
        #print(hg.shape)
        #print(h[0].shape)
        #hg = h[g.nodes[0].data['center_node'].detach().numpy()[0]]
        #print(hg.shape)
        return hg, self.classify(hg)

In [6]:
from torch.utils.data import DataLoader
import torch.optim as optim
import pickle
path = './data/cora/'
with open(path + 'list_subgraph.pkl', 'rb') as f:
    total_subgraph = pickle.load(f)
    
with open(path + 'label.pkl', 'rb') as f:
    info = pickle.load(f)

with open(path + 'center.pkl', 'rb') as f:
    center_node = pickle.load(f)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
class Classifier_Finetune(nn.Module):
    def __init__(self, hidden_dim, n_classes):
        super(Classifier_Finetune, self).__init__()

        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, model, g, to_fetch, features):
        
        h, _ = model(g, to_fetch, features)
        
        return self.classify(h)

In [12]:
def run(fold_n):
    path = './data/cora/'

    path = path + 'fold'+str(fold_n)+'/'
    trainset = Subgraphs(path, 'train', total_subgraph, info, center_node)
    valset = Subgraphs(path, 'val', total_subgraph, info, center_node)
    testset = Subgraphs_test(path, 'test', total_subgraph, info, center_node, 1, 12)

    data_loader_test = DataLoader(testset, batch_size=1, shuffle=True, collate_fn=collate_test)

    data_loader = DataLoader(trainset, batch_size=64, shuffle=True, collate_fn=collate)
    
    model = Classifier(1433, 256, 7)
    model.to(device)
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    model.train()
    
    print('model training')
    for epoch in range(10):
        epoch_loss = 0
        for iter, (bg, label, to_fetch) in enumerate(data_loader):
            bg = bg.to(device)
            label = label.to(device)
            features = bg.ndata['h']
            hid, prediction = model(bg, to_fetch, features)
            #print(prediction.shape)
            loss = loss_func(prediction, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.detach().item()
        epoch_loss /= (iter + 1)
        
        if epoch%1 ==0:
            print('Epoch {}, training loss {:.4f}'.format(epoch, epoch_loss))
    
    print('model finetuning')
    accs = []
    for iter, (bg, label, to_fetch, bg_qry, label_qry, to_fetch_qry) in enumerate(data_loader_test):
        model_finetune = Classifier_Finetune(256, 2)
        to_fetch = torch.LongTensor(to_fetch.reshape(-1,))
        label = label.reshape(-1,)

        to_fetch_qry = torch.LongTensor(to_fetch_qry.reshape(-1,))
        label_qry = label_qry.reshape(-1,)    

        update_step = 10
        loss_steps = []
        acc_steps = []

        for i in range(update_step):
            model_finetune.train()
            bg_ = copy.deepcopy(bg)
            features = bg_.ndata['h']
            prediction = model_finetune(model, bg_, to_fetch, features)
            loss = loss_func(prediction, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            model_finetune.eval()
            bg_ = copy.deepcopy(bg_qry)
            features = bg_.ndata['h']
            pred_qry = model_finetune(model, bg_, to_fetch_qry, features)
            probs_Y = torch.softmax(pred_qry, 1)
            argmax_Y = torch.max(probs_Y, 1)[1].view(-1, 1)

            label_q = torch.tensor(label_qry).float().view(-1, 1)
            acc = (label_q == argmax_Y.detach().cpu().float()).sum().item() / len(label_q) * 100

            acc_steps.append(acc)
            loss_steps.append(loss.item())
        accs.append(acc_steps)
        if iter%20 == 0:
            print('step ', iter)
    print(np.mean(np.array(accs), 0))
    
    
    print('model KNN')
    accs = []
    for iter, (bg, label, to_fetch, bg_qry, label_qry, to_fetch_qry) in enumerate(data_loader_test):
        to_fetch = torch.LongTensor(to_fetch.reshape(-1,))
        label = label.reshape(-1,)
        
        to_fetch_qry = torch.LongTensor(to_fetch_qry.reshape(-1,))
        label_qry = label_qry.reshape(-1,) 
        
        bg_ = copy.deepcopy(bg)
        features = bg_.ndata['h']
        h, prediction = model(bg_, to_fetch, features)
        
        bg_ = copy.deepcopy(bg_qry)
        features = bg_.ndata['h']
        h_qry, pred_qry = model(bg_, to_fetch_qry, features)
        
        from sklearn.metrics import pairwise_distances
        
        dist = pairwise_distances(h.detach().numpy(), h_qry.detach().numpy())
        
        y_pred = np.argmin(dist, 0).reshape(-1,)
        
        label_dict = dict(zip(label.numpy(), list(range(label.numpy().reshape(-1,).shape[0]))))
        y_pred = np.array([label_dict[i] for i in y_pred])
        #print(y_pred.shape)
        label_q = label_qry.float().view(-1, ).numpy()

        acc = sum(label_q == y_pred) / len(label_q) * 100

        accs.append(acc)
        
        if iter%20 == 0:
            print('step ', iter)
            
    print(np.mean(np.array(accs)))

In [13]:
run(1)

model training
Epoch 0, training loss 1.2025
Epoch 1, training loss 0.7275
Epoch 2, training loss 0.4294
Epoch 3, training loss 0.2130
Epoch 4, training loss 0.0939
Epoch 5, training loss 0.0591
Epoch 6, training loss 0.0457
Epoch 7, training loss 0.0470
Epoch 8, training loss 0.0324
Epoch 9, training loss 0.0296
model finetuning


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  0
step  20
step  40
step  60
step  80
[51.25       51.79166667 54.25       56.58333333 60.33333333 64.20833333
 67.04166667 70.25       75.04166667 78.625     ]
model KNN
step  0
step  20
step  40
step  60
step  80
90.37500000000001


In [ ]:
run(2)

In [ ]:
run(3)

In [ ]:
run(4)

In [ ]:
run(5)